# Perform Retraining of DAGMNet

## Load Model

In [2]:
import scipy
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import tensorflow
import random

In [3]:
DAGMNet_ch2_name =  os.path.join("D:\ADS_Algorithm\ADSv1.3\data\Trained_Nets", 'DAGMNet_CH2.h5')

In [4]:
DAGMNet_ch2 = tensorflow.keras.models.load_model(DAGMNet_ch2_name, compile=False)

In [5]:
DAGMNet_ch2.input

<KerasTensor: shape=(None, 96, 112, 48, 2) dtype=float32 (created by layer 'input_1')>

In [26]:
swi_sample_img = nib.load("D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\SWI_Train\\2018-104_01-10113-D0MR_9_Ax_T2_GRE__SkullStripped_Training_Normalized.nii.gz")
swi_sample_data = swi_sample_img.get_fdata()
tof_sample_img = nib.load("D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\TOF3D_Train\\2018-104_01-10113-D0MR_6_3D_TOF_LARGE__SkullStripped_Training_Normalized.nii.gz")
tof_sample_data = tof_sample_img.get_fdata()

In [27]:
swi_sample_data = swi_sample_data[:,:,:,np.newaxis]
tof_sample_data = tof_sample_data[:,:,:,np.newaxis]

In [28]:
dagmnet_input = np.expand_dims(np.concatenate((swi_sample_data,tof_sample_data),axis=3), axis=0)

In [23]:
dagmnet_output = DAGMNet_ch2.predict(dagmnet_input)

1/1 [==============================] - 0s 39ms/step


## Read-in Training Images

In [30]:
swi_dir = "D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\SWI_Train"
tof_dir = "D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\TOF3D_Train"

swi_files = [file for file in os.listdir(swi_dir) if file.endswith('.nii.gz')]
tof_files = [file for file in os.listdir(tof_dir) if file.endswith('.nii.gz')]

X_train = np.zeros((len(swi_files), 1, 96, 112, 48, 2))
index = 0

for swi_file, tof_file in zip(swi_files,tof_files):
    swi_img = nib.load(os.path.join(swi_dir, swi_file))
    swi_data = swi_img.get_fdata()
    tof_img = nib.load(os.path.join(tof_dir, tof_file))
    tof_data = swi_img.get_fdata()

    swi_data = swi_data[:,:,:,np.newaxis]
    tof_data = tof_data[:,:,:,np.newaxis]
    input_array = np.expand_dims(np.concatenate((swi_data,tof_data),axis=3), axis=0)
    X_train[index] = input_array
    index += 1

In [33]:
mask_dir = "D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\MASK_Train"

mask_files = [file for file in os.listdir(mask_dir) if file.endswith('.nii.gz')]

y_train = np.zeros((len(mask_files), 96, 112, 48))
index = 0

for mask_file in mask_files:
    mask_img = nib.load(os.path.join(mask_dir, mask_file))
    mask_data = mask_img.get_fdata()
    y_train[index] = mask_data
    index += 1

## Separate Test Batch of Images

In [8]:
mask_train_dir = "D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\MASK_Train"
mask_test_dir = "D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\MASK_Test"
swi_train_dir = "D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\SWI_Train"
swi_test_dir = "D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\SWI_Test"
tof_train_dir = "D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\TOF3D_Train"
tof_test_dir = "D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\TOF3D_Test"

In [9]:
random.seed(777)

In [10]:
test_indexes = random.sample(range(len(os.listdir(mask_train_dir))), 100)

In [11]:
file_list = [os.listdir(mask_train_dir)[index] for index in test_indexes]
for file in file_list:
    os.rename(os.path.join(mask_train_dir, file), os.path.join(mask_test_dir, file))

In [12]:
file_list = [os.listdir(swi_train_dir)[index] for index in test_indexes]
for file in file_list:
    os.rename(os.path.join(swi_train_dir, file), os.path.join(swi_test_dir, file))

In [13]:
file_list = [os.listdir(tof_train_dir)[index] for index in test_indexes]
for file in file_list:
    os.rename(os.path.join(tof_train_dir, file), os.path.join(tof_test_dir, file))

## Define Custom Loss Functions

In [ ]:
def l1_loss(y_pred):
    np.sum()

In [ ]:
def combined_loss(y_true, y_pred):
    cross_entropy_loss = tensorflow.keras.losses.BinaryCrossEntropy()
    dice_loss = tensorflow.keras.losses.Dice()